First two cells are the ways that I generated the lookup table and got the list of all the relevant neighbors so that I can use that for the adjacency matrix populating

In [1]:
#not_relevant: this is written as a function, but not really intended to be used multiple times, just functionalized to put it to the side when I run
def producing_uniprot_ensembl_mapping():

    #new version using the downloaded mapping file:
    # Read the tab-delimited file with 22 columns
    import pandas as pd
    df = pd.read_csv(r"C:\Users\abdul\OneDrive - University of Cambridge\Desktop\MDRA\cellbox_torch\Abdullah_kuziez\Testing_encodings\HUMAN_9606_idmapping_selected.tab", 
                    sep='\t',
                    header=None,
                    names=[f'col_{i}' for i in range(22)],
                    low_memory=False)
    df_uniprots_and_ensembles=df[['col_0','col_18','col_19','col_20']]
    del df
    # Rename columns to match the provided names
    df_uniprots_and_ensembles.rename(columns={
        'col_0': 'Uniprot.ID',
        'col_18': 'ENSG',
        'col_19': 'ENST', 
        'col_20': 'ENSP',
    }, inplace=True)


    from tqdm import tqdm

    for index in tqdm(range(len(df_uniprots_and_ensembles['ENSP'])), desc="Processing ENSP entries"):
        ID_list=df_uniprots_and_ensembles['ENSP'][index]
        # Split the string of IDs by semicolon and strip whitespace
        if isinstance(ID_list, str):
            split_ids = [id_.strip() for id_ in ID_list.split(';')]
        else:
            split_ids = []
        # For each element in split_ids, remove the '.' and everything after the '.':
        split_ids = [id_.split('.', 1)[0] for id_ in split_ids]
        df_uniprots_and_ensembles['ENSP'][index]=split_ids

    df_uniprots_and_ensembles.to_csv('uniprots_to_ensembls_lookup_table.csv')
    return df_uniprots_and_ensembles




#NOTE  #general testing that I did to make sure it was working:
# import pandas as pd

# import ast
# df_uniprots_and_ensembles2 = pd.read_csv(
#     'uniprots_to_ensembls_lookup_table.csv',
#     converters={'ENSP': lambda x: ast.literal_eval(x) if pd.notnull(x) else []}
# )
# df_uniprots_and_ensembles2=df_uniprots_and_ensembles2[['Uniprot.ID','ENSG','ENST','ENSP']]
# # more testing:
# ENSMBL_list=['ENSP00000438677','ENSP00000298316','ENSP00000005257']
# # For each ensembl ID in ENSMBL_list, search the 'ENSP' column (which contains lists of strings) for a match.
# matching_uniprot_ids = []
# for ens_id in ENSMBL_list:
#     # Find rows where the ENSP list contains the ens_id
#     matches = df_uniprots_and_ensembles2[df_uniprots_and_ensembles2['ENSP'].apply(lambda x: ens_id in x if isinstance(x, list) else False)]
#     # For each match, get the Uniprot.ID (could be multiple per ENSP)
#     for _, row in matches.iterrows():
#         matching_uniprot_ids.append(row['Uniprot.ID'])

# print("Matching Uniprot IDs for given ENSMBL_list:", matching_uniprot_ids)






In [2]:
#not relevant, but this is how I generated the original list of neighbors and wrote to file, functionalized so it doesnt run always
def get_and_write_neighbor_list():
    import numpy as np
    import pandas as pd
    #get starting data
    data_path=r'C:\Users\abdul\OneDrive - University of Cambridge\Desktop\MDRA\cellbox_torch\preprocessing\data.csv'
    large_dataset=pd.read_csv(data_path)
    targeted_proteins_with_metadata=pd.read_csv('targeted_proteins_with_metadata.csv')

    #get the zeroth order proteins
    listed_zeroth_order_prots=get_clean_unique_uniprots(targeted_proteins_with_metadata)
    zeroth_order_proteins=get_uniprots_in_dataset(listed_zeroth_order_prots,large_dataset)
    print("number of zeroth order proteins: ",len(zeroth_order_proteins))

    # #get the first order proteins by finding the partners of the zeroth order proteins
    first_order_proteins_ENSMBL=get_protein_partners(zeroth_order_proteins)
    first_order_proteins_uniprot=ensmbl_to_uniprot_local(first_order_proteins_ENSMBL)
    print("total number of first order proteins: ",len(first_order_proteins_uniprot))
    final_first_order_proteins=get_uniprots_in_dataset(list(set(first_order_proteins_uniprot)),large_dataset)
    print("number of first order proteins in the dataset: ",len(final_first_order_proteins))

    #get the second order proteins by finding the partners of the first order proteins
    second_order_proteins_ENSMBL=get_protein_partners(final_first_order_proteins)
    second_order_proteins_uniprot=ensmbl_to_uniprot_local(second_order_proteins_ENSMBL)
    print("total number of second order proteins: ",len(second_order_proteins_uniprot))
    final_second_order_proteins=get_uniprots_in_dataset(list(set(second_order_proteins_uniprot)),large_dataset)
    print("number of second order proteins in the dataset: ",len(final_second_order_proteins))


    # Create a DataFrame with the three columns
    # First ensure all arrays are the same length by padding with None
    max_length = max(len(zeroth_order_proteins), len(final_first_order_proteins), len(final_second_order_proteins))
    zeroth_order_padded = zeroth_order_proteins + [None] * (max_length - len(zeroth_order_proteins))
    first_order_padded = final_first_order_proteins + [None] * (max_length - len(final_first_order_proteins))
    second_order_padded = final_second_order_proteins + [None] * (max_length - len(final_second_order_proteins))
    protein_orders_df = pd.DataFrame({
        'zeroth_order': zeroth_order_padded,
        'first_order': first_order_padded,
        'second_order': second_order_padded
    })

    # Save to CSV
    protein_orders_df.to_csv('protein_neighbors_and_degrees.csv', index=False)
    return



The older functions are preserved for posterity because I might want to mine them and get useful code at some point from them

In [3]:
#older functions that were writtien to directly mark neighbors
import pandas as pd
import re

def binary_marking_first__order(targeted_proteins_with_metadata):
    #first make a dataframe that is all zeros of the same dimension as the targeted proteins with metadata
    targeted_proteins_matrix = pd.DataFrame(0, index=targeted_proteins_with_metadata.index, columns=targeted_proteins_with_metadata.columns)
    #using targeted proteins with metadata, identify for each row which proteins are targeted;
    for index, row in targeted_proteins_with_metadata.iterrows():
        targets = row['Uniprot.ID']
        if isinstance(targets, str):
            # Use regex to split on either semicolon, pipe, or comma
            target_list = [t.strip().upper() for t in re.split('[;|,]', targets)]
            # Mark these targets as 1 in the matrix
            for target in target_list:
                if target in targeted_proteins_matrix.columns:
                    targeted_proteins_matrix.loc[index, target] = 1
    return targeted_proteins_matrix

def continuous_marking_first__order(targeted_proteins_with_metadata):
    #first make a dataframe that is all zeros of the same dimension as the targeted proteins with metadata
    targeted_proteins_matrix = pd.DataFrame(0, index=targeted_proteins_with_metadata.index, columns=targeted_proteins_with_metadata.columns)
    #using targeted proteins with metadata, identify for each row which proteins are targeted;
    for index, row in targeted_proteins_with_metadata.iterrows():
        targets = row['Uniprot.ID']
        if isinstance(targets, str):
            # Use regex to split on either semicolon, pipe, or comma
            target_list = [t.strip().upper() for t in re.split('[;|,]', targets)]
            # Mark these targets as 1 in the matrix
            for target in target_list:
                if target in targeted_proteins_matrix.columns:
                    targeted_proteins_matrix.loc[index, target] = targeted_proteins_with_metadata.loc[index, target]*1 #the one is for first order interactions\
    return targeted_proteins_matrix
#unclear if this is working correctly;

def binary_marking_second_order_neighbors(first_order_marked): #TODO write this method, but first decide on a database to use
    # we should do this for each of the rows in first order marked independently
    #first make a copy of the first order marked
    second_order_marked = first_order_marked.copy()
    #then iterate through each row in the second order marked
    for index, row in second_order_marked.iterrows():
        #get the names of the columns that have a 1 in them
        first_order_proteins = row[row == 1].index.tolist()
        # then use the get protein partners to get the second order neighbors of the proteins that have a one in them
        print('Index: ', index, 'Proteins: ', first_order_proteins)
        second_order_neighbors = get_protein_partners(first_order_proteins)
        #then use the convert to uniprot to get the uniprot ID's of the second order neighbors
        second_order_uniprot_neighbors = ensmbl_to_uniprot_local(second_order_neighbors)
        #then go through and check the columns to see if any of the second order neighbors are in the columns, and if so, add a .33 for the second order neighbors
        for neighbor in second_order_uniprot_neighbors:
            if neighbor in second_order_marked.columns:
                if second_order_marked.loc[index, neighbor] == 0:
                    second_order_marked.loc[index, neighbor] = 0.33
    return second_order_marked


# def continuous_marking_second_order_neighbors(first_order_marked,prot_info): #TODO write this method, but first decide on a database to use

def filter_out_all_zero_rows(dataset):
    # Filter out rows where all values are 0
    filtered_dataset = dataset.loc[~(dataset == 0).all(axis=1)]
    return filtered_dataset



In [4]:
#deprecated, using ENSMBL to Uniprot using online service
def convert_ensembl_to_uniprot_online(ensembl_ids):
    """
takes in a list of ensemble ID's and returns a list of uniprot ID's using
 the uniprot mapping service
    """
    import unipressed
    if len(ensembl_ids) == 0:
        return []
    
    job = unipressed.IdMappingClient.submit(
        'Ensembl_Protein',   # source database
        'UniProtKB',         # target database
        ids=ensembl_ids      # list of Ensembl protein IDs
    )
    
    uniprot_ids = []
    time.sleep(.5)
    results = unipressed.id_mapping.core.IdMappingJob.each_result(job)
    print(results)
    for result in results:
        uniprot_ids.append(result['to'])
        
    return uniprot_ids

These functions are used in the first two cells to generate the lookup table and the list of partners. They are also used to get the adjacency matrix

In [5]:
#Functions to do ENSMBL to uniprot
def get_ENSMBL_Uniprot_lookup_table(file_path):
    """imports lookup table and returns it
    lookup table was made using producing_uniprot_ensmbl_mapping """
    import ast, pandas as pd
    df_uniprots_and_ensembles = pd.read_csv(
    file_path,
    converters={'ENSP': lambda x: ast.literal_eval(x) if pd.notnull(x) else []}
)
    df_uniprots_and_ensembles=df_uniprots_and_ensembles[['Uniprot.ID','ENSG','ENST','ENSP']]
    return df_uniprots_and_ensembles

def ensmbl_to_uniprot_local(ENSMBLs,df_uniprots_and_ensembles):
    """intakes a list of ENSMBLs or a single string of ENSMBLS and the lookup table,
    returns the Uniprot IDs for each of the ENSMBLS"""
    matching_uniprot_ids=[]
    if isinstance(ENSMBLs,list):
        for ens_id in ENSMBLs:
    # Find rows where the ENSP list contains the ens_id
            matches = df_uniprots_and_ensembles[df_uniprots_and_ensembles['ENSP'].apply(lambda x: ens_id in x if isinstance(x, list) else False)]
    # For each match, get the Uniprot.ID (could be multiple per ENSP)
            for _, row in matches.iterrows():

                matching_uniprot_ids.append(row['Uniprot.ID'])
            
    if isinstance(ENSMBLs, str):
        # Check each row's ENSP list for a string that matches ENSMBLs
        matches = df_uniprots_and_ensembles[df_uniprots_and_ensembles['ENSP'].apply(lambda x: ENSMBLs in x if isinstance(x, list) else False)]
        for _, row in matches.iterrows():
            matching_uniprot_ids.append(row['Uniprot.ID'])

    return matching_uniprot_ids




In [6]:
#fxn to get the protein partners from STRING
def get_protein_partners(protein_id, min_score=900):
    """
    Get protein interaction partners from STRING database
    
    Args:
        protein_id (str): UniProt accession number
        min_score (int): Minimum interaction score (0-1000)
        
    Returns:
        list: List of STRING IDs for interaction partners (without species prefix)
    """
    #if null exit    
    if len(protein_id) == 0:
            return []
    #for each protein in the list get the partners
    overall_partners=[]
    for protein in protein_id:
        url = ("https://string-db.org/api/tsv/interaction_partners"
            f"?identifiers={protein}&species=9606&required_score={min_score}")
        
        data = requests.get(url).text
        df = pd.read_csv(io.StringIO(data), sep='\t')
        # Remove the '9606.' prefix from each STRING ID
        partners = [partner.replace('9606.', '') for partner in df['stringId_B'].tolist()]
        overall_partners.append(partners)
    flat_partners=[]
    for i in overall_partners:
         for j in i:
              flat_partners.append(j)

    return flat_partners


In [7]:
#batched protein partners, all of them
def get_protein_partners_batched(protein_ids, min_score=900, batch_size=100):
    if len(protein_ids) == 0:
        return []
    overall_partners = []
    string_a=[]
    string_b=[]
    prots=[]
    # Process proteins in batches
    for i in range(0, len(protein_ids), batch_size):
        batch = protein_ids[i:i + batch_size]
        # Join proteins with %0d separator
        batch_str = '%0d'.join(batch)
        
        url = ("https://string-db.org/api/tsv/interaction_partners"
               f"?identifiers={batch_str}&species=9606&required_score={min_score}")
        
        data = requests.get(url).text
        df = pd.read_csv(io.StringIO(data), sep='\t')
        string_a.extend(df['stringId_A'].tolist())
        string_b.extend(df['stringId_B'].tolist())
    
    # Group by stringA and aggregate stringB into lists
    fixed_string_b=[]
    fixed_string_a=[]
    for string_b_item in string_b:
         fixed_string_b.append(string_b_item.replace('9606.',''))
    for string_a_item in string_a:
        fixed_string_a.append(string_a_item.replace('9606.',''))
    

    overall_df_2=pd.DataFrame({'stringId_A':fixed_string_a,'stringId_B':fixed_string_b})
    
    restructured_df = overall_df_2.groupby('stringId_A')['stringId_B'].agg(list).reset_index()
    # Remove the '9606.' prefix from each STRING ID
    return restructured_df


In [8]:
#fxns to get the original uinque uniprots in dataset and then used in neighbor finding
def get_clean_unique_uniprots(targeted_proteins_with_metadata):
    """takes in a dataframe outputs a list of unique uniprots found in the dataframe Uniprot.ID column
     splits things using regX, flattens, then gets unique elements """
    uniprots=targeted_proteins_with_metadata['Uniprot.ID']
    cleaned_uniprots=[]
    for IDs in uniprots:
        if isinstance(IDs, str) and IDs !=' not protein' and IDs !='not human':
            indiv_ID=[t.strip().upper() for t in re.split('[;|,]', IDs)]
            cleaned_uniprots.append(indiv_ID)

    flat_uniprots=get_flattened_list(cleaned_uniprots)
    unique_uniprots=list(set(flat_uniprots)) #gives the unique elements of the set
    return unique_uniprots

def get_flattened_list(list_of_lists):
    """takes in a list of lists and returns a list of all the elements in the lists"""
    flat_list=[]
    for i in list_of_lists:
        for j in i:
            flat_list.append(j)
    return flat_list

#gets the uniprots in the dataset
def get_uniprots_in_dataset(protein_list,dataset):
    """takes in a list of uniprots and a dataset and returns a list of uniprots that are in the dataset"""
    proteins_in_dataset=[]
    columns=dataset.columns
    for i in protein_list:
        if i in columns:
            proteins_in_dataset.append(i)
    return list(set(proteins_in_dataset))


In [9]:
def ensmbl_to_uniprot_local(ENSMBLs, df_uniprots_and_ensembles):
    """
    Optimized version of ensmbl_to_uniprot_local that uses vectorized operations
    """
    if not ENSMBLs:  # Handle empty input
        return []
    
    # Convert ENSMBLs to a set for faster lookups
    if isinstance(ENSMBLs, str):
        ensembl_set = {ENSMBLs}
    else:
        ensembl_set = set(ENSMBLs)
    
    # Create a mapping of ENSP to Uniprot.ID
    # First, explode the ENSP lists into separate rows
    exploded_df = df_uniprots_and_ensembles.explode('ENSP')
    
    # Filter only the rows where ENSP is in our target set
    matches = exploded_df[exploded_df['ENSP'].isin(ensembl_set)]
    
    # Get unique Uniprot IDs
    matching_uniprot_ids = matches['Uniprot.ID'].unique().tolist()
    
    return matching_uniprot_ids

In [10]:
#rewriting the making of the adjacency matrix:
#intaking relevant files:
import pandas as pd
import requests
import csv
import io
import unipressed
import unipressed.id_mapping
import time
import re
from tqdm import tqdm
df_uniprots_and_ensembles=get_ENSMBL_Uniprot_lookup_table('uniprots_to_ensembls_lookup_table.csv')
df_neighbors=pd.read_csv('protein_orders.csv')
protein_list=df_neighbors['second_order']

#now for each row, go through and mark 1st and second degree neighbors

pts=get_protein_partners_batched(protein_list,batch_size=500)
#affixing on the uniprot ID's
uniprot_ids=[]
for id in pts['stringId_A']:
    #pull them out of individual lists and turn them into strings
    uniprot_ids.append(ensmbl_to_uniprot_local(id,df_uniprots_and_ensembles)[0])
pts['uniprots']=uniprot_ids
partner_list=pts





In [11]:
#local implementation of get protein partner
def get_protein_partner_local(protein,parnter_list=partner_list):
    # Find the row where the uniprots column matches the input protein
    matching_row = partner_list[partner_list['uniprots'] == protein]
    
    # If a match is found, return the stringId_B value
    if not matching_row.empty:
        return matching_row['stringId_B'].tolist()[0]
    else:
        return None
    
test=list(set(protein_list))

In [12]:
# # Get unique ENSEMBL IDs from partner_list
# unique_ensembl_ids = set()
# for partners in partner_list['stringId_B']:
#     if isinstance(partners, list):
#         unique_ensembl_ids.update(partners)
#     else:
#         unique_ensembl_ids.add(partners)

# # Filter df_uniprots_and_ensembles to only include rows where ensp is in unique_ensembl_ids
# filtered_df = df_uniprots_and_ensembles[df_uniprots_and_ensembles['ensp'].isin(unique_ensembl_ids)]

In [16]:
# building adjacency matrix
# Restrict the lookup table to only the Uniprot IDs present in df_neighbors['second_order']
# First, get the set of Uniprot IDs in df_neighbors
uniprots_in_neighbors = set(df_neighbors['second_order'].unique())
# Filter the lookup table to only those Uniprot IDs
restricted_df_uniprots_and_ensembles = df_uniprots_and_ensembles[df_uniprots_and_ensembles['Uniprot.ID'].isin(uniprots_in_neighbors)].reset_index(drop=True)


#start building the adjacency matrix:
adjacency_matrix = pd.DataFrame(0, 
                            index=protein_list,
                            columns=protein_list)

                        #first degree neighbors
for protein in tqdm(adjacency_matrix.index, desc="Processing proteins"):
                   
                    #STEP 1: mark first degree neighbors
    partners_ensembl = get_protein_partner_local(protein) #working
    partners_uniprot = ensmbl_to_uniprot_local(partners_ensembl, restricted_df_uniprots_and_ensembles) # not working
  
    # now make sure that the partners are in the matrix
    partners_uniprot_in_matrix = get_uniprots_in_dataset(partners_uniprot, adjacency_matrix)

    # then update the matrix
    adjacency_matrix.loc[protein, partners_uniprot_in_matrix] = 1

    #                 #second degree neighbors
    for j in partners_uniprot_in_matrix: #change it to partners_uniprot if you want to include links that skip outside the observed protein dataset

                    #STEP 2: mark second_degree_neighbors
        partners_of_partners_ensembl=list(set(get_protein_partner_local(j)))
        partners_of_partners_uniprot=list(set(ensmbl_to_uniprot_local(partners_of_partners_ensembl,restricted_df_uniprots_and_ensembles)))

        partners_of_partners_uniprot_in_matrix=list(set(get_uniprots_in_dataset(partners_of_partners_uniprot,adjacency_matrix)))
        #we need to be careful to not overwrite the ones
        for k in partners_of_partners_uniprot_in_matrix:
            if adjacency_matrix.loc[protein,k]==0:
                adjacency_matrix.loc[protein,k]=.33 


#NOTE!! THere is a meaningful question to ask here as follows: Assume: Protein A and C in dataset, B is not
#if protein A is directly targeted, and B is the first degree neighbor of A and C is the first degree neighbor of B, should I mark C as the second degree neighbor of A?
#if I do, I preserve the logical link, but B is not in the dataset. This is a meaningful question!
#at present, implemented such that we generate second degree neigbhors by only looking at first degree neighbors in the dataset.


Processing proteins:   0%|          | 0/928 [00:00<?, ?it/s]

Processing proteins: 100%|██████████| 928/928 [00:50<00:00, 18.22it/s]


In [17]:
adjacency_matrix.to_csv('adjacency_matrix.csv')

In [12]:
#constructing an alternae way of protein protein interaction by evaluating similarity of the vectors for each protein.
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances

adjacency_matrix=pd.read_csv('adjacency_matrix.csv')
pre_matrix=adjacency_matrix.iloc[:,1:].to_numpy()
labels=adjacency_matrix.iloc[:,0].to_numpy()
#using cosine similarity:
cosine_similarity=1-pairwise_distances(pre_matrix,metric='cosine')

#using pairwise distances:
jaccard_similarity=1-pairwise_distances(pre_matrix,metric='jaccard')

c:\Users\abdul\.conda\envs\cellbox\lib\site-packages\sklearn\metrics\pairwise.py:2182: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


In [37]:
#now I have to draw the rest of the owl :( plug in the adjacency matrix flat into cellbox and see if it works.
import sys
import os
#what this code is doing is adding the cellbox directory to the path and then I can import the cellbox package which now meansd I can use the code there

sys.path.append('../../../cellbox')
print(os.getcwd())
import cellbox


c:\Users\abdul\OneDrive - University of Cambridge\Desktop\MDRA\cellbox_torch\Abdullah_kuziez\Testing_encodings\String_adjacency
   _____     _ _ ____              
  / ____|   | | |  _ \             
 | |     ___| | | |_) | _____  __  
 | |    / _ \ | |  _ < / _ \ \/ /  
 | |___|  __/ | | |_) | (_) >  <   
  \_____\___|_|_|____/ \___/_/\_\  
Running CellBox scripts developed in Sander lab
Maintained by Bo Yuan, Judy Shen, and Augustin Luna; contributions by Phuc Nguyen

        version 0.1.0
        -- Aug 24, 2023 --
        * First stable release of CellBox pytorch
        * Add pytest with a range of test cases
        
Tutorials and documentations are available at https://github.com/Mustardburger/CellBox
If you want to discuss the usage or to report a bug, please use the 'Issues' function at GitHub.
If you find CellBox useful for your research, please consider citing the corresponding publication.
For more information, please email us at boyuan@g.harvard.edu and c_shen@g.harvard.ed

In [40]:
from cellbox.config import Config
from cellbox.model_torch import factory

args = Config("running_adacency_config.json")
model, args = factory(args)

# Prepare your input data as needed, then:
# output = model(y0, mu)  # y0 and mu are your input tensors

AttributeError: 'Config' object has no attribute 'iter_train'

Doing model training now

In [ ]:
# providing the learning vectors now:
import torch
from sklearn.model_selection import train_test_split

# Convert ground truth to PyTorch tensor
ground_truth_tensor = torch.tensor(ground_truth.values, dtype=torch.float32)

# Split the data into training and test sets (70:30 split)
train_data, test_data = train_test_split(ground_truth_tensor, test_size=0.3, random_state=42)

print(f"Training data shape: {train_data.shape}")
print(f"Test data shape: {test_data.shape}")
#now that we have the training and test data split we can start learning




Training data shape: torch.Size([21, 61])
Test data shape: torch.Size([10, 61])


In [6]:
#GENERAL NOTES

#figuring out how to actually make the model;
#principle component analysis to reduce dimensionality
#collaborative filtering algorithm to interpolate missing values
#netflix style problem;
#

#now do the machine learning...
#the ground truths are targeted_proteins_with_metadata; but we will cleave off the metadata
#the input is the first order marked.

In [10]:
# Legacy does not work

# import numpy as np
# targeted_proteins_with_metadata=pd.read_csv('targeted_proteins_with_metadata.csv')
# # get the



# first_order_marked=binary_marking_first__order(targeted_proteins_with_metadata)
# protein = "P38398"          # replace with your UniProt accession # need to actually update this:
# partners=(get_protein_partners(protein))
# print(partners)
# print(convert_ensembl_to_uniprot(partners))
# second_order_marked=binary_marking_second_order_neighbors(first_order_marked)

# filtered_network_vectors=filter_out_all_zero_rows(second_order_marked)
# filtered_network_vectors=filtered_network_vectors.iloc[:,:-12]
# rows_to_keep=filtered_network_vectors.index.tolist()

# #now make the ground truth matrix
# ground_truth=targeted_proteins_with_metadata.iloc[:,:-12]
# ground_truth=ground_truth.iloc[rows_to_keep,:]
# ground_truth = ground_truth.fillna(0)

['ENSP00000255416', 'ENSP00000263321', 'ENSP00000273588', 'ENSP00000296589', 'ENSP00000373570', 'ENSP00000341550', 'ENSP00000319531', 'ENSP00000451605', 'ENSP00000370737', 'ENSP00000363046', 'ENSP00000287996', 'ENSP00000361064', 'ENSP00000333262', 'ENSP00000260386', 'ENSP00000376142', 'ENSP00000411152', 'ENSP00000263370', 'ENSP00000413937', 'ENSP00000074304', 'ENSP00000363046', 'ENSP00000287996', 'ENSP00000361064', 'ENSP00000333262', 'ENSP00000260386', 'ENSP00000376142', 'ENSP00000411152', 'ENSP00000263370', 'ENSP00000413937', 'ENSP00000074304', 'ENSP00000440689', 'ENSP00000342007', 'ENSP00000301141', 'ENSP00000286479', 'ENSP00000353720', 'ENSP00000368727', 'ENSP00000225275']
<generator object IdMappingJob.each_result at 0x000001F0EDD464A0>
['Q13203', 'P14679', 'P48728', 'Q9UMX9', 'P17643', 'Q71RS6', 'P23434', 'Q01726', 'P23378', 'Q8NFU5', 'Q9H8X2', 'Q9UNW1', 'Q15735', 'P23677', 'P49441', 'P27987', 'Q96DU7', 'Q9BT40', 'P05181', 'P05177', 'P11509', 'P11245', 'P23141', 'P47989', 'P05164'

KeyboardInterrupt: 